In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy

df = pd.read_csv('../data/industrial-energy-tariffs.csv')
df.shape



In [ ]:
# subset it to 1000 rows
df = df.sample(n=100).reset_index(drop=True)
# find nan values
# notes_idx = ~df["Notes"].isnull().values
# df_filter = df[notes_idx]

In [ ]:

elec_x = []
elec_y = []
demand_x = []
demand_y = []

# look at each set of tariffs in the dataframa by looking for the marker of the start of each batch: a value in the 'Notes' column, and within each batch sorting by electric and demand 'type' finding the max and min for each
i = 0
while i < len(df):
    if pd.notnull(df.loc[i, 'Notes']):
        elec_max = 0
        elec_min = numpy.inf
        demand_max = 0
        demand_min = numpy.inf
        i += 1
        while pd.isnull(df.loc[i, 'Notes']) and i < len(df):
            if df.loc[i, 'type'] == 'electric':
                if df.loc[i, 'charge (imperial)'] > elec_max:
                    elec_max = df.loc[i, 'charge (imperial)']
                if df.loc[i, 'charge (imperial)'] < elec_min:
                    elec_min = df.loc[i, 'charge (imperial)']
            elif df.loc[i, 'type'] == 'demand':
                if df.loc[i, 'charge (imperial)'] > demand_max:
                    demand_max = df.loc[i, 'charge (imperial)']
                if df.loc[i, 'charge (imperial)'] < demand_min:
                    demand_min = df.loc[i, 'charge (imperial)']
            else:
                continue
            i +=1
        if elec_min != numpy.inf and elec_max != 0:
            elec_x.append(elec_min)
            elec_y.append(elec_max / elec_min)
        if demand_min != numpy.inf and demand_max != 0:
            demand_x.append(demand_min)
            demand_y.append(demand_max / demand_min)
    i += 1

print(elec_y)
plt.scatter(elec_x, elec_y)
plt.scatter(demand_x, demand_y)

In [ ]:
elec_tou = 0
elec_tiered = 0
elec_both = 0
dem_tou = 0
dem_tiered = 0
dem_both = 0
elec_none = 0
dem_none = 0


# look at each set of tariffs in the dataframa by looking for the marker of the start of each batch: a value in the 'Notes' column, and within each batch sorting by electric and demand 'type' finding the max and min for each
i = 0
while i < len(df):
    if pd.notnull(df.loc[i, 'Notes']):
        elec_isTOU = False
        elec_isTiered = False
        demand_isTOU = False
        demand_isTiered = False
        i += 1
        while i < len(df) and pd.isnull(df.loc[i, 'Notes']):
            if df.loc[i, 'type'] == 'electric':
                if df.loc[i, 'basic_charge_limit (imperial)'] != 0 and pd.notnull(df.loc[i, 'basic_charge_limit (imperial)']):
                    elec_isTiered = True
                if df.loc[i, 'hour_start'] != 0 and df.loc[i, 'hour_end'] != 23 and pd.notnull(df.loc[i, 'hour_start']) and pd.notnull(df.loc[i, 'hour_end']):
                    elec_isTOU = True
            if df.loc[i, 'type'] == 'demand':
                if df.loc[i, 'basic_charge_limit (imperial)'] != 0 and pd.notnull(df.loc[i, 'basic_charge_limit (imperial)']):
                    demand_isTiered = True
                if df.loc[i, 'hour_start'] != 0 and df.loc[i, 'hour_end'] != 23 and pd.notnull(df.loc[i, 'hour_start']) and pd.notnull(df.loc[i, 'hour_end']):
                    demand_isTOU = True
            i +=1
        if elec_isTOU and elec_isTiered:
            elec_both += 1
        elif elec_isTOU:
            elec_tou += 1
        elif elec_isTiered:
            elec_tiered += 1
        else:
            elec_none += 1
        
        if demand_isTOU and demand_isTiered:
            dem_both += 1
        elif demand_isTOU:
            dem_tou += 1
        elif demand_isTiered:
            dem_tiered += 1
        else:
            dem_none += 1

        if i <len(df) and pd.notnull(df.loc[i, 'Notes']):
            continue
    i += 1

print(f"elec both: {elec_both}, elec tou: {elec_tou}, elec tiered: {elec_tiered}, elec none: {elec_none}")
print(f"dem both: {dem_both}, dem tou: {dem_tou}, dem tiered: {dem_tiered}, dem none: {dem_none}")
print(f"elec total: {elec_both + elec_tou + elec_tiered + elec_none}, dem total: {dem_both + dem_tou + dem_tiered + dem_none}")

In [ ]:
# Assuming you have a dictionary with the data
data = {
    'TOU': elec_tou,  # replace with actual data
    'Tiered': elec_tiered,  # replace with actual data
    'Both': elec_both,  # replace with actual data
    'Flat': elec_none  # replace with actual data
}

# Create the bar graph
plt.figure(figsize=(10, 5), dpi=300)
plt.bar(data.keys(), data.values(), color='#8C1515')
plt.title('Electricity Charges')
plt.xlabel('Tariff Type')
plt.ylabel('Number of Tariffs')
plt.show()

In [ ]:
# Assuming you have a dictionary with the demand data
demand_data = {
    'TOU': dem_tou,  # replace with actual data
    'Tiered': dem_tiered,  # replace with actual data
    'Both': dem_both,  # replace with actual data
    'Flat': dem_none  # replace with actual data
}

# Create the bar graph for demand data
plt.figure(figsize=(10, 5), dpi=300)
plt.bar(demand_data.keys(), demand_data.values(), color='#8C1515')
plt.title('Demand Charges')
plt.xlabel('Tariff Type')
plt.ylabel('Number of Tariffs')
plt.show()

In [ ]:
print(df.loc[0, 'Notes'])

In [ ]:
import pgeocode
df = pd.read_csv('../data/zipcodes.csv', header=None)
array = df[0].tolist()
array.pop(0)

In [ ]:
# Initialize the pgeocode object for US
nomi = pgeocode.Nominatim('US')

def get_state_from_zip(zipcode):
    """
    Takes a zip code and returns the corresponding state abbreviation.
    """
    location = nomi.query_postal_code(zipcode)
    return location.state_code

def count_zipcodes_by_state(zipcode_list):
    """
    Takes a list of zip codes and returns a dictionary with the state abbreviation as the key
    and the number of zip codes in that state as the value.
    """
    state_count = {}

    for zipcode in zipcode_list:
        state = get_state_from_zip(zipcode)
        
        if state:  # If the state is valid (not NaN or None)
            if state in state_count:
                state_count[state] += 1
            else:
                state_count[state] = 1
    
    return state_count

# Example list of zip codes
zipcodes = array

# Generate the dictionary with the count of zip codes per state
state_zip_count = count_zipcodes_by_state(zipcodes)

# Print the result
print(state_zip_count)

In [ ]:
import folium
import geopandas as gpd
import matplotlib.pyplot as plt
from matplotlib.colors import Normalize
from matplotlib import cm
state_data = state_zip_count

In [ ]:
gdf = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))
us_states = gdf[gdf['iso_a2'] == 'US']

# Normalize the values for coloring
norm = Normalize(vmin=min(state_data.values()), vmax=max(state_data.values()))
cmap = cm.ScalarMappable(norm=norm, cmap='YlOrRd')

# Function to get the color based on magnitude
def get_color(state):
    if state in state_data:
        return cmap.to_rgba(state_data[state])
    else:
        return (0, 0, 0, 0)  # Transparent if the state is not in the data

# Create a folium map
m = folium.Map(location=[37.8, -96], zoom_start=4)

# Add states to the map
# for _, state in us_states.iterrows():
#     folium.GeoJson(
#         state['geometry'],
#         style_function=lambda feature, state=state: {
#             'fillColor': plt.colors.to_hex(get_color(state['name'])),
#             'color': 'black',
#             'weight': 2,

In [ ]:
import geodatasets

# Load the US states shapefile from GeoPandas
# You can also download a shapefile of US states if needed
# For example: gpd.read_file("path_to_your_shapefile.shp")
us_states = gpd.read_file(os.getcwd()+'/cb_2018_us_state_500k')
us_states.head()

# Filter to only include US states
us_states = us_states[us_states['continent'] == 'North America']

# Create a new column in the GeoDataFrame to hold the shading values
us_states['value'] = us_states['iso_a2'].map(state_data)

# Plotting the choropleth
fig, ax = plt.subplots(1, 1, figsize=(10, 6))
us_states.boundary.plot(ax=ax, linewidth=1)
us_states.plot(column='value', ax=ax, legend=True,
               cmap='OrRd',  # Choose your color map
               missing_kwds={'color': 'lightgrey', 'label': 'Missing values'})

# Customize the plot
ax.set_title('Choropleth Map of US States')
ax.set_xlabel('Longitude')
ax.set_ylabel('Latitude')

plt.show()

In [ ]:
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt
# Load the shapefile
us = gpd.read_file('../data/cb_2018_us_state_500k')

# Define a list of valid state abbreviations
valid_states = [
    'AL', 'AK', 'AZ', 'AR', 'CA', 'CO', 'CT', 'DE', 'FL', 'GA', 
    'HI', 'ID', 'IL', 'IN', 'IA', 'KS', 'KY', 'LA', 'ME', 'MD', 
    'MA', 'MI', 'MN', 'MS', 'MO', 'MT', 'NE', 'NV', 'NH', 'NJ', 
    'NM', 'NY', 'NC', 'ND', 'OH', 'OK', 'OR', 'PA', 'RI', 'SC', 
    'SD', 'TN', 'TX', 'UT', 'VT', 'VA', 'WA', 'WV', 'WI', 'WY'
]

# Filter the GeoDataFrame to keep only states
us_states = us[us['STUSPS'].isin(valid_states)]

# Convert the state_data dictionary to a DataFrame
data_df = pd.DataFrame(list(state_data.items()), columns=['STUSPS', 'value'])

# Merge the state data with the GeoDataFrame
merged = us_states.merge(data_df, on='STUSPS', how='left')

# Create a figure and axis
fig, ax = plt.subplots(1, 1, figsize=(15, 10), dpi=300)

# Plot the boundaries and the choropleth with black edges
merged.boundary.plot(ax=ax, linewidth=1, edgecolor='black')
merged.plot(column='value', ax=ax,
            cmap='OrRd',
            edgecolor='black',  # Set edges to black
            missing_kwds={
                'color': 'lightgrey',
                'edgecolor': 'black',
                'label': 'No data'
            })

# Create a colorbar on the right
sm = plt.cm.ScalarMappable(cmap='OrRd', norm=plt.Normalize(vmin=merged['value'].min(), vmax=merged['value'].max()))
sm.set_array([])
cbar = fig.colorbar(sm, ax=ax, orientation='vertical', pad=0.01, shrink=0.763)  # Adjust shrink value
cbar.set_label("Shading Value by State")

# Set limits to zoom into the contiguous U.S.
ax.set_xlim(-128, -64)  # Adjust the x-limits as needed
ax.set_ylim(23, 51)     # Adjust the y-limits as needed

plt.title('Zipcode Coverage by State')
plt.show()
